# Advanced decorators

## Plan

1. Decorator recap.
2. Decorators with arguments.
3. Classes as decorators.
4. Decorating classes.

## Reference material: https://decorators.mathspp.com/intro.html

In [1]:
import functools

def decorator(function_to_decorate):  # 1
    @functools.wraps(function_to_decorate)  # 3
    # 2                # 4
    def inner_function(*args, **kwargs):
        ...  # 5
        result = function_to_decorate(*args, **kwargs)  # 6
        ...  # 7
        return result  # 8

    return inner_function  # 9

In [5]:
def decorator(f):
    def wrapper(*args, **kwargs):
        ...
        result = f(*args, **kwargs)
        ...
        return result

    return wrapper

@decorator
def foo():
    pass

# same as

def foo():
    """Docstring."""
    pass

foo = decorator(foo)

In [4]:
?foo

Signature: foo(*args, **kwargs)
Docstring: <no docstring>
File:      /var/folders/29/cpfnqrmx0ll8m1vp9f9fmnx00000gn/T/ipykernel_10836/2138163077.py
Type:      function

In [9]:
from functools import wraps

def decorator(f):
    @wraps(f)
    def wrapper(*args, **kwargs):
        ...
        result = f(*args, **kwargs)
        ...
        return result

    return wrapper

@decorator
def foo():
    """Docstring"""
    pass

?foo

Signature: foo()
Docstring: Docstring
File:      /var/folders/29/cpfnqrmx0ll8m1vp9f9fmnx00000gn/T/ipykernel_10836/1764871718.py
Type:      function

In [21]:
import time

def cache(f):
    _cache = {}
    def cached(*args):
        if args not in _cache:
            _cache[args] = f(*args)
        return _cache[args]

    return cached


@cache
def f():
    time.sleep(1)
    return 2

# Decorators make use “closures”

In [26]:
f.__closure__

(<cell at 0x103f8f640: dict object at 0x104742180>,
 <cell at 0x103f8ef50: function object at 0x10470db20>)

In [27]:
f.__closure__[0]

<cell at 0x103f8f640: dict object at 0x104742180>

In [28]:
f.__closure__[0].cell_contents

{(): 2}

In [29]:
@cache
def add(a, b):
    return a + b

In [32]:
add(1, 2); add(3, 4)
print(add.__closure__[0].cell_contents)

{(1, 2): 3, (3, 4): 7}


In [33]:
from functools import lru_cache

# lru_cache adds a cache to your function

## The cache may be bounded

In [34]:
@lru_cache(1024)
def add(a, b):
    return a + b

## How do you create decorators with arguments?!

In [43]:
# Cache with a maximum size.

MAX_SIZE = 2

def cache(f):
    _cache = {}
    def cached(*args):
        if args not in _cache:
            _cache[args] = f(*args)
        if len(_cache) > MAX_SIZE:
            key_to_remove = next(iter(_cache.keys()))
            del _cache[key_to_remove]
        
        return _cache[args]

    return cached

In [44]:
@cache
def add(a, b):
    return a + b

In [45]:
add(1, 2)
add(3, 4)
add(5, 6)

11

# To create a decorator with arguments:

 - Indent the decorator without arguments & the global variable
 - The global variable becomes an argument of the outermost function
 - Return what was the outermost function (which is now the intermediate function)

In [47]:
# Cache with a maximum size.

def cache(max_size):
    
    def cache_inner(f):
        _cache = {}
        def cached(*args):
            if args not in _cache:
                _cache[args] = f(*args)
            
            if len(_cache) > max_size:
                key_to_remove = next(iter(_cache.keys()))
                del _cache[key_to_remove]
            
            return _cache[args]
    
        return cached

    return cache_inner

In [48]:
@cache(3)
def add(a, b):
    return a + b

In [53]:
# Cache with a maximum size.

def lru_cache(max_size):
    
    def cache_inner(f):
        _cache = {}  # The (key, value) pairs in the dictionary
        # will be kept in order of recent usage.
        def cached(*args):
            if args not in _cache:
                _cache[args] = f(*args)
            else:  # Ensure this cached value is marked
                # as having been used recently.
                value = _cache[args]
                del _cache[args]
                _cache[args] = value

            if len(_cache) > max_size:
                key_to_remove = next(iter(_cache.keys()))
                del _cache[key_to_remove]
            
            return _cache[args]
    
        return cached

    return cache_inner

# Classes as decorators

In [54]:
from functools import cache, lru_cache

In [55]:
cache

<function functools.cache(user_function, /)>

In [56]:
lru_cache

<function functools.lru_cache(maxsize=128, typed=False)>

In [57]:
@cache
def foo():
    pass

In [60]:
foo.cache_info()

CacheInfo(hits=0, misses=0, maxsize=None, currsize=0)

## Classes as decorators allow you to add more flexible functionality to your functions

In [64]:
class Decorator:
    def __init__(self, function):
        self._f = function

In [65]:
@Decorator
def foo():
    print("Hello, world!")

In [68]:
foo  # foo is now an instance of the class Decorator

In [69]:
# I should be able to use the function:
foo()

TypeError: 'Decorator' object is not callable